# REFINITIV EIKON - CONSTITUENTS AND FRAGMENTATION 

In [ ]:
import eikon as ek
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# SET VARIABLES

In [ ]:
# GERMAN MARKET SPECIFIC
index = "0#.GDAXI"
home_id = "d."

In [ ]:
# WHICH MTFs
biggest_mtfs = [".DE", ".DXE", ".TQE", ".AQE"]

# SET APP KEY

In [ ]:
app_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
ek.set_app_key(app_key)

In [ ]:
ek.set_timeout(1e6)

# MAKE AND SET PATH TO SAVE

In [ ]:
# folder_name = Path(f"CON_&_FRA")
# folder_name.mkdir(parents=True, exist_ok=True) 

In [ ]:
# path_to_save = f"./CON_&_FRA/"

# GET CONSTITUENTS

In [ ]:
index_constituents, _ = ek.get_data(index, fields = ["TR.CompanyName"])
index_constituents.to_csv(path_to_save + "index_constituents" + ".csv")
index_constituents

In [ ]:
index_constituents.to_pickle("index_constituents")

In [ ]:
index_constituents = pd.read_pickle("index_constituents")
index_constituents

# GET FRAGMENTATION

In [ ]:
fragmentation = pd.DataFrame()

for ric in index_constituents["Instrument"]:
    main_ric_base = ric.split(".")[0]

    isin = ek.get_symbology(symbol = ric, to_symbol_type = "ISIN")
    isin = isin.loc[:,"ISIN"][0]

    fragmentation_i = ek.get_symbology(symbol = isin, from_symbol_type = "ISIN", to_symbol_type = "RIC", best_match = False)

    all_rics = fragmentation_i["RICs"][0]
    
    cleaned_rics = [item for item in all_rics if (item.split(".")[0] == main_ric_base and ("^" not in item) and ("xbo" not in item) and ("xt" not in item)) or (home_id in item and ("^" not in item))]
    cleaned_rics = [item for item in cleaned_rics if any(item.endswith(mtf) for mtf in biggest_mtfs)]

    fragmentation_i = pd.DataFrame({"ric": ric, "fragmentation": cleaned_rics})
    fragmentation = pd.concat([fragmentation, fragmentation_i], ignore_index=True)

In [ ]:
fragmentation["venue"] = fragmentation["fragmentation"].apply(lambda x: x.split(".")[1])

In [ ]:
fragmentation["ric"].value_counts()

In [ ]:
fragmentation["venue"].value_counts()

In [ ]:
# fragmentation_cross = pd.crosstab(fragmentation["ric"], fragmentation["venue"])
# fragmentation_cross.to_csv(path_to_save + "fragmentation" + ".csv")
# pd.crosstab(fragmentation["ric"], fragmentation["venue"])

In [ ]:
fragmentation.to_pickle("fragmentation")

In [ ]:
fragmentation = pd.read_pickle("fragmentation")
fragmentation